In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import dspy
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, MIPROv2
from dspy.evaluate import Evaluate
import asyncio
import os
import numpy as np
from dotenv import load_dotenv
from datasets import load_dataset
import logging
import pickle

from nano_graphrag._utils import compute_mdhash_id
from nano_graphrag.entity_extraction.extract import generate_dataset
from nano_graphrag.entity_extraction.module_typed import TypedEntityRelationshipExtractor
from nano_graphrag.entity_extraction.metric import relationship_similarity_metric, entity_recall_metric

/opt/homebrew/Caskroom/miniconda/base/envs/nano-graphrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
WORKING_DIR = "./nano_graphrag_cache_finetune_entity_relationship_dspy"

load_dotenv()

logging.basicConfig(level=logging.WARNING)
logging.getLogger("nano-graphrag").setLevel(logging.DEBUG)

np.random.seed(1337)

In [4]:
system_prompt = """
    You are a world-class AI system, capable of complex reasoning and reflection. 
    Reason through the query, and then provide your final response. 
    If you detect that you made a mistake in your reasoning at any point, correct yourself.
    Think carefully.
"""
lm = dspy.OpenAI(
    model="deepseek-chat", 
    model_type="chat", 
    api_key=os.environ["DEEPSEEK_API_KEY"], 
    base_url=os.environ["DEEPSEEK_BASE_URL"], 
    system_prompt=system_prompt, 
    temperature=1.0,
    top_p=1.0,
    max_tokens=4096
)
llama = dspy.OllamaLocal(
    model="llama3.1", 
    model_type="chat",
    system=system_prompt,
    temperature=1.0,
    top_p=1.0,
    max_tokens=4096
)
dspy.settings.configure(lm=lm, experimental=True)

In [5]:
os.makedirs(WORKING_DIR, exist_ok=True)
entity_relationship_rs_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news_rs.json")
entity_relationship_miprov2_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news_miprov2.json")
entity_relationship_trainset_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news_trainset.pkl")
entity_relationship_valset_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news_valset.pkl")
entity_relationship_devset_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news_devset.pkl")

trainset = pickle.load(open(entity_relationship_trainset_path, "rb"))
valset = pickle.load(open(entity_relationship_valset_path, "rb"))
devset = pickle.load(open(entity_relationship_devset_path, "rb"))

In [6]:
trainset = [example for example in trainset if len(example.relationships) > 0 and len(example.entities) > 0]
trainset

[Example({'input_text': "As students from Marjory Stoneman Douglas High School confront lawmakers with demands to restrict sales of assault rifles, there were warnings by the president of the Florida school administration association that schools in the state were vulnerable to such an attack.\nHighlights\n“Nikolas Cruz was able to purchase an assault rifle before he was able to buy a beer,” said Stoneman Douglas student Laurenzo Prado, referring to a Florida law that allows people as young as 18 to buy assault weapons. Students galvanized by the deadly mass shooting at the Florida high school confronted lawmakers with demands to restrict sales of assault rifles, while President Donald Trump suggested arming teachers as a way to stop more U.S. rampages.\nTwo weeks before a gunman fatally shot 17 people at a Florida high school, Bill Lee, the president of the state’s school administrators association, warned that Florida’s schools were vulnerable to just such an attack. “It’s not a matt

In [7]:
for example in trainset:
    for relationship in example.relationships:
        if relationship['order'] == 2:
            print(relationship)

{'src_id': 'GENERAL ELECTRIC', 'tgt_id': 'WARREN BUFFETT', 'description': "Warren Buffett may comment on General Electric's performance in his annual letter to Berkshire Hathaway shareholders.", 'weight': 0.6, 'order': 2}
{'src_id': 'U.S. FED', 'tgt_id': 'WARREN BUFFETT', 'description': "Warren Buffett may address the U.S. Fed's interest rate decisions in his annual letter.", 'weight': 0.6, 'order': 2}
{'src_id': 'HONG KONG NEWSROOM', 'tgt_id': 'SHANGHAI STOCK EXCHANGE', 'description': 'The Hong Kong newsroom reported the block trade filed at the Shanghai Stock Exchange.', 'weight': 0.6, 'order': 2}
{'src_id': '5G-READY DEVICES', 'tgt_id': 'MASS-MARKET DEPLOYMENTS', 'description': 'Mass-market deployments will follow the availability of 5G-ready devices.', 'weight': 0.6, 'order': 2}
{'src_id': 'FDA 510(K) CLEARANCE', 'tgt_id': 'LAPAROSCOPIC INGUINAL HERNIA', 'description': 'The FDA 510(k) clearance includes expanded indications for the Senhance Surgical System, specifically for laparos

In [8]:
for example in trainset:
    for relationship in example.relationships:
        if relationship['order'] == 3:
            print(relationship)

In [9]:
trainset[0].relationships[:2]

[{'src_id': 'MARJORY STONEMAN DOUGLAS HIGH SCHOOL',
  'tgt_id': 'NIKOLAS CRUZ',
  'description': 'Nikolas Cruz was involved in a deadly mass shooting at Marjory Stoneman Douglas High School.',
  'weight': 0.8,
  'order': 1},
 {'src_id': 'MARJORY STONEMAN DOUGLAS HIGH SCHOOL',
  'tgt_id': 'LAURENZO PRADO',
  'description': 'Laurenzo Prado is a student at Marjory Stoneman Douglas High School who spoke about the shooting.',
  'weight': 0.7,
  'order': 1}]

In [7]:
valset = [example for example in valset if len(example.relationships) > 0 and len(example.entities) > 0]
valset

[Example({'input_text': '0 COMMENTS Global stock markets were mixed Friday, with many recovering from declines a day earlier even as investors continued to weigh up the impact of higher inflation and interest rates on equities.\nThe Stoxx Europe 600 was roughly flat around noon in Europe, while most indexes in the Asia-Pacific region were higher.\nGermany’s DAX and France’s CAC 40 rose 0.8% and 1.4% this week, respectively, but a 1.5% fall in Italy’s FTSE MIB weighed on the broader European index.\nU.S. stocks were projected to open higher, with the S&P 500 and Dow Jones Industrial Average futures contracts up 0.2% and 0.4%, respectively. On Thursday, U.S. stocks rose to pare the week’s losses .\nAs 10-year U.S. Treasury yields have ticked higher this year, equity investors are watching bond markets closely for signs that higher yields are affecting corporate performance.\n“Historically, the probability of loss for the S&P 500 increases when the 10-year Treasury yield rises above 3%,” 

In [11]:
valset[0].relationships[:2]

[{'src_id': 'GLOBAL STOCK MARKETS',
  'tgt_id': 'STOXX EUROPE 600',
  'description': 'The Stoxx Europe 600 is a part of the global stock markets, representing European stocks.',
  'weight': 0.8,
  'order': 1},
 {'src_id': 'GLOBAL STOCK MARKETS',
  'tgt_id': 'ASIA-PACIFIC REGION',
  'description': 'The Asia-Pacific region is a part of the global stock markets, where most indexes were higher on Friday.',
  'weight': 0.7,
  'order': 1}]

In [12]:
for example in valset:
    for relationship in example.relationships:
        if relationship['order'] == 2:
            print(relationship)

{'src_id': 'GERMAN GOVERNMENT BOND YIELDS', 'tgt_id': '10-YEAR U.S. TREASURY YIELDS', 'description': 'German government bond yields and 10-year U.S. Treasury yields are both affected by global economic conditions, with German yields falling as U.S. yields rise.', 'weight': 0.7, 'order': 2}


In [13]:
for example in valset:
    for relationship in example.relationships:
        if relationship['order'] == 3:
            print(relationship)

In [8]:
devset = [example for example in devset if len(example.relationships) > 0 and len(example.entities) > 0]
devset

[Example({'input_text': 'Kendrick Lamar’s Pulitzer Prize win is the latest sign of the growing recognition of hip-hop—this time from one of America’s highest-profile cultural institutions.\nThe rapper, who won for his album “DAMN.,” is the first winner who isn’t a classical or jazz artist since the first Pulitzer for music was issued in 1943. Aside from previous award winners Wynton Marsalis, Henry Threadgill and Ornette Coleman, it has largely been a prize for classical composers.\n... To Read the Full Story Subscribe Sign In', 'entities': [{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': "A renowned rapper who won the Pulitzer Prize for his album 'DAMN.'. He is the first non-classical or jazz artist to win the Pulitzer for music since its inception in 1943.", 'importance_score': 0.9}, {'entity_name': 'PULITZER PRIZE', 'entity_type': 'AWARD', 'description': "A prestigious award in the United States, given in various categories including journalism, literature,

In [15]:
devset[0].relationships[:2]

[{'src_id': 'KENDRICK LAMAR',
  'tgt_id': 'PULITZER PRIZE',
  'description': "Kendrick Lamar won the Pulitzer Prize for music for his album 'DAMN.'. This win is significant as it marks the first time a non-classical or jazz artist has won the award since 1943.",
  'weight': 0.8,
  'order': 1},
 {'src_id': 'KENDRICK LAMAR',
  'tgt_id': 'DAMN.',
  'description': "Kendrick Lamar is the artist behind the album 'DAMN.', which won the Pulitzer Prize for music.",
  'weight': 0.9,
  'order': 1}]

In [16]:
for example in devset:
    for relationship in example.relationships:
        if relationship['order'] == 2:
            print(relationship)

{'src_id': 'CLOUD KINGS', 'tgt_id': 'FANG', 'description': "The collapse of the 'cloud kings' could have implications for FANG companies, as they are often seen as bellwethers for the tech sector.", 'weight': 0.7, 'order': 2}


In [17]:
for example in devset:
    for relationship in example.relationships:
        if relationship['order'] == 3:
            print(relationship)

In [9]:
model = TypedEntityRelationshipExtractor()
model

extractor.predictor = Predict(StringSignature(input_text, entity_types -> reasoning, entities_relationships
    instructions='Given a text document that is potentially relevant to this activity and a list of entity types, \nidentify all entities of those types from the text and all relationships among the identified entities.\n\nFor entities:\n    - Each entity name should be an actual atomic word from the input text. Avoid duplicates and generic terms.\n    - Make sure descriptions are detailed and comprehensive, including:\n        1. The entity\'s role or significance in the context\n        2. Key attributes or characteristics\n        3. Relationships to other entities (if applicable)\n        4. Historical or cultural relevance (if applicable)\n        5. Any notable actions or events associated with the entity\n    - All entity types from the text must be included. \n    - IMPORTANT: Only use entity types from the provided \'entity_types\' list. Do not introduce new entity types

In [19]:
metrics = [entity_recall_metric, relationship_similarity_metric]
for metric in metrics:
    evaluate = Evaluate(
        devset=devset, 
        metric=metric, 
        num_threads=os.cpu_count(), 
        display_progress=True,
        display_table=5,
    )
    evaluate(model)

Average Metric: 3.6384559884559886 / 5  (72.8): 100%|██████████| 5/5 [00:00<00:00, 305.79it/s]
INFO:dspy.evaluate.evaluate:2024-09-19T06:27:15.314677Z [info     ] Average Metric: 3.6384559884559886 / 5 (72.8%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=218


,input_text,example_entities,example_relationships,pred_entities,pred_relationships,entity_recall_metric
0,"Kendrick Lamar’s Pulitzer Prize win is the latest sign of the growing recognition of hip-hop—this time from one of America’s highest-profile cultural institutions. The rapper,...","[{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': ""A renowned rapper who won the Pulitzer Prize for his album 'DAMN.'. He is the first non-classical or jazz...","[{'src_id': 'KENDRICK LAMAR', 'tgt_id': 'PULITZER PRIZE', 'description': ""Kendrick Lamar won the Pulitzer Prize for music for his album 'DAMN.'. This win is significant as it...","[{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': ""A renowned rapper who won the Pulitzer Prize for his album 'DAMN.'. He is the first non-classical or jazz...","[{'src_id': 'KENDRICK LAMAR', 'tgt_id': 'PULITZER PRIZE', 'description': ""Kendrick Lamar won the Pulitzer Prize for his album 'DAMN.', marking a significant recognition of hip-hop music by...",✔️ [0.8571428571428571]
1,Cramer Remix: The fall of cloud stocks could be a worrisome sign for tech 11 Hours Ago Jim Cramer deciphers the collapse of the “cloud...,"[{'entity_name': 'CRAMER', 'entity_type': 'PERSON', 'description': ""Jim Cramer, a well-known financial commentator and host of 'Mad Money' on CNBC, who is analyzing the fall of cloud...","[{'src_id': 'CRAMER', 'tgt_id': 'CLOUD STOCKS', 'description': 'Jim Cramer is analyzing the fall of cloud stocks, providing insights into what this decline could mean for the...","[{'entity_name': 'JIM CRAMER', 'entity_type': 'PERSON', 'description': 'Jim Cramer is a financial commentator who deciphers the collapse of cloud stocks and its potential impact on FANG...","[{'src_id': 'JIM CRAMER', 'tgt_id': 'CLOUD STOCKS', 'description': 'Jim Cramer deciphers the collapse of cloud stocks, providing insights into their significance and potential impact.', 'weight': 0.8,...",✔️ [0.4]
2,* Mulberry manufactures 55 pct of leather goods in UK * Import cost of raw materials has gone up after Brexit vote * Brand fully...,"[{'entity_name': 'MULBERRY', 'entity_type': 'ORGANIZATION', 'description': 'A handbag maker committed to manufacturing in Britain, facing higher leather import costs post-Brexit. The company has two manufacturing plants...","[{'src_id': 'MULBERRY', 'tgt_id': 'UK', 'description': 'Mulberry manufactures 55% of its leather goods in the UK and remains committed to the country despite post-Brexit challenges.', 'weight':...","[{'entity_name': 'MULBERRY', 'entity_type': 'ORGANIZATION', 'description': 'Mulberry is a handbag maker committed to manufacturing in Britain. It produces about 55 percent of its leather goods in...","[{'src_id': 'MULBERRY', 'tgt_id': 'UK', 'description': 'Mulberry is committed to manufacturing in the UK, with about 55 percent of its leather goods produced there.', 'weight': 1.0,...",✔️ [0.9090909090909091]
3,"COLUMBIA, Md., April 25, 2018 (GLOBE NEWSWIRE) -- W. R. Grace & Co. (NYSE:GRA) today announced that it has declared a quarterly cash dividend of...","[{'entity_name': 'W. R. GRACE & CO.', 'entity_type': 'ORGANIZATION', 'description': 'A leading global supplier of catalysts and engineered materials, with two industry-leading business segments: Catalysts Technologies...","[{'src_id': 'W. R. GRACE & CO.', 'tgt_id': 'NYSE', 'description': 'W. R. Grace & Co. is listed on the New York Stock Exchange under the symbol...","[{'entity_name': 'W. R. GRACE & CO.', 'entity_type': 'ORGANIZATION', 'description': 'W. R. Grace & Co. is a leading global supplier of catalysts and engineered materials. It...","[{'src_id': 'W. R. GRACE & CO.', 'tgt_id': 'RICH BADMINGTON', 'description': 'Rich Badmington is in media relations for W. R. Grace & Co., indicating a professional...",✔️ [0.5833333333333334]
4,"Top marketer joins company to lead global marketing and brand strategies FRAMINGHAM, Mass. & DUBLIN--(BUSINESS WIRE)-- Globoforce ® , a leading provider

DEBUG:nano-graphrag:Entities: 10 | Direct Relationships: 9 | Second-order: 0 | Third-order: 0 | Total Relationships: 9
DEBUG:nano-graphrag:Entities: 4 | Direct Relationships: 2 | Second-order: 1 | Third-order: 0 | Total Relationships: 3
DEBUG:nano-graphrag:Entities: 19 | Direct Relationships: 18 | Second-order: 0 | Third-order: 0 | Total Relationships: 18
  0%|          | 0/5 [00:00<?, ?it/s]DEBUG:nano-graphrag:Entities: 8 | Direct Relationships: 6 | Second-order: 0 | Third-order: 0 | Total Relationships: 6
DEBUG:nano-graphrag:Entities: 8 | Direct Relationships: 7 | Second-order: 0 | Third-order: 0 | Total Relationships: 7
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:http

,input_text,example_entities,example_relationships,pred_entities,pred_relationships,relationship_similarity_metric
0,"Kendrick Lamar’s Pulitzer Prize win is the latest sign of the growing recognition of hip-hop—this time from one of America’s highest-profile cultural institutions. The rapper,...","[{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': ""A renowned rapper who won the Pulitzer Prize for his album 'DAMN.'. He is the first non-classical or jazz...","[{'src_id': 'KENDRICK LAMAR', 'tgt_id': 'PULITZER PRIZE', 'description': ""Kendrick Lamar won the Pulitzer Prize for music for his album 'DAMN.'. This win is significant as it...","[{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': ""A renowned rapper who won the Pulitzer Prize for his album 'DAMN.'. He is the first non-classical or jazz...","[{'src_id': 'KENDRICK LAMAR', 'tgt_id': 'PULITZER PRIZE', 'description': ""Kendrick Lamar won the Pulitzer Prize for his album 'DAMN.', marking a significant recognition of hip-hop music by...",✔️ [0.2833333333333334]
1,Cramer Remix: The fall of cloud stocks could be a worrisome sign for tech 11 Hours Ago Jim Cramer deciphers the collapse of the “cloud...,"[{'entity_name': 'CRAMER', 'entity_type': 'PERSON', 'description': ""Jim Cramer, a well-known financial commentator and host of 'Mad Money' on CNBC, who is analyzing the fall of cloud...","[{'src_id': 'CRAMER', 'tgt_id': 'CLOUD STOCKS', 'description': 'Jim Cramer is analyzing the fall of cloud stocks, providing insights into what this decline could mean for the...","[{'entity_name': 'JIM CRAMER', 'entity_type': 'PERSON', 'description': 'Jim Cramer is a financial commentator who deciphers the collapse of cloud stocks and its potential impact on FANG...","[{'src_id': 'JIM CRAMER', 'tgt_id': 'CLOUD STOCKS', 'description': 'Jim Cramer deciphers the collapse of cloud stocks, providing insights into their significance and potential impact.', 'weight': 0.8,...",✔️ [0.6666666666666666]
2,* Mulberry manufactures 55 pct of leather goods in UK * Import cost of raw materials has gone up after Brexit vote * Brand fully...,"[{'entity_name': 'MULBERRY', 'entity_type': 'ORGANIZATION', 'description': 'A handbag maker committed to manufacturing in Britain, facing higher leather import costs post-Brexit. The company has two manufacturing plants...","[{'src_id': 'MULBERRY', 'tgt_id': 'UK', 'description': 'Mulberry manufactures 55% of its leather goods in the UK and remains committed to the country despite post-Brexit challenges.', 'weight':...","[{'entity_name': 'MULBERRY', 'entity_type': 'ORGANIZATION', 'description': 'Mulberry is a handbag maker committed to manufacturing in Britain. It produces about 55 percent of its leather goods in...","[{'src_id': 'MULBERRY', 'tgt_id': 'UK', 'description': 'Mulberry is committed to manufacturing in the UK, with about 55 percent of its leather goods produced there.', 'weight': 1.0,...",✔️ [0.375]
3,"COLUMBIA, Md., April 25, 2018 (GLOBE NEWSWIRE) -- W. R. Grace & Co. (NYSE:GRA) today announced that it has declared a quarterly cash dividend of...","[{'entity_name': 'W. R. GRACE & CO.', 'entity_type': 'ORGANIZATION', 'description': 'A leading global supplier of catalysts and engineered materials, with two industry-leading business segments: Catalysts Technologies...","[{'src_id': 'W. R. GRACE & CO.', 'tgt_id': 'NYSE', 'description': 'W. R. Grace & Co. is listed on the New York Stock Exchange under the symbol...","[{'entity_name': 'W. R. GRACE & CO.', 'entity_type': 'ORGANIZATION', 'description': 'W. R. Grace & Co. is a leading global supplier of catalysts and engineered materials. It...","[{'src_id': 'W. R. GRACE & CO.', 'tgt_id': 'RICH BADMINGTON', 'description': 'Rich Badmington is in media relations for W. R. Grace & Co., indicating a professional...",✔️ [0.16666666666666666]
4,"Top marketer joins company to lead global marketing and brand strategies FRAMINGHAM, Mass. & DUBLIN--(BUSINESS WIRE)-- Globoforce ® , a lea

In [20]:
optimizer = BootstrapFewShotWithRandomSearch(
    metric=relationship_similarity_metric, 
    num_threads=os.cpu_count(),
    num_candidate_programs=10,
    max_labeled_demos=8,
    max_bootstrapped_demos=4,
)
rs_model = optimizer.compile(model, trainset=trainset, valset=valset)
rs_model

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 10 candidate sets.


  0%|          | 0/3 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


KeyboardInterrupt: 

In [ ]:
metrics = [entity_recall_metric, relationship_similarity_metric]
for metric in metrics:
    evaluate = Evaluate(
        devset=devset, 
        metric=metric, 
        num_threads=os.cpu_count(), 
        display_progress=True,
        display_table=5,
    )
    evaluate(rs_model)

In [ ]:
rs_model.save(entity_relationship_rs_path)

In [10]:
optimizer = MIPROv2(
    prompt_model=lm,
    task_model=lm,
    metric=entity_recall_metric,
    init_temperature=1.4,
    num_candidates=10,
    verbose=True
)
kwargs = dict(num_threads=os.cpu_count(), display_progress=True, display_table=0)
miprov2_model = optimizer.compile(
    model, 
    trainset=trainset, 
    valset=valset, 
    requires_permission_to_run=False,
    num_batches=10, 
    max_labeled_demos=8, 
    max_bootstrapped_demos=4, 
    eval_kwargs=kwargs
)
miprov2_model


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 25 examples in minibatch * 10 batches + 19 examples in train set * 1 full evals = 269 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may c

 21%|██        | 4/19 [04:32<17:02, 68.14s/it]


Bootstrapped 4 full traces after 5 examples in round 0.


 21%|██        | 4/19 [05:25<20:22, 81.47s/it] 


Bootstrapped 4 full traces after 5 examples in round 0.


...

Average Metric: 13.920678590415433 / 19  (73.3): 100%|██████████| 19/19 [00:00<00:00, 3211.69it/s]
INFO:dspy.evaluate.evaluate:2024-09-19T07:40:22.629245Z [info     ] Average Metric: 13.920678590415433 / 19 (73.3%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=218


FULL TRACE


Average Metric: 0.6666666666666666 / 1  (66.7): 100%|██████████| 1/1 [00:00<00:00, 1475.31it/s]
INFO:dspy.evaluate.evaluate:2024-09-19T07:40:22.639496Z [info     ] Average Metric: 0.6666666666666666 / 1 (66.7%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=218
[I 2024-09-19 03:40:22,645] Trial 9 finished with value: 73.27 and parameters: {'0_predictor_instruction': 9, '0_predictor_demos': 7}. Best is trial 0 with value: 80.08.





Given a text document that is potentially relevant to this activity and a list of entity types, 
identify all entities of those types from the text and all relationships among the identified entities.

For entities:
    - Each entity name should be an actual atomic word from the input text. Avoid duplicates and generic terms.
    - Make sure descriptions are detailed and comprehensive, including:
        1. The entity's role or significance in the context
        2. Key attributes or characteristics
        3. Relationships to other entities (if applicable)
        4. Historical or cultural relevance (if applicable)
        5. Any notable actions or events associated with the entity
    - All entity types from the text must be included. 
    - IMPORTANT: Only use entity types from the provided 'entity_types' list. Do not introduce new entity types.

For relationships:
    - Make sure relationship descriptions are detailed and comprehensive, including:
        1. The nature of the re

extractor.predictor = Predict(StringSignature(input_text, entity_types -> reasoning, entities_relationships
    instructions='Given a text document and a list of entity types, meticulously identify all entities of the specified types from the text. For each entity, provide a comprehensive description that includes its role, key attributes, relationships to other entities, historical or cultural relevance, and notable actions or events. Ensure that only the provided entity types are used and avoid duplicates. For relationships, describe their nature, impact, historical context, evolution, and notable events. Include direct and higher-order relationships up to the third order, ensuring that "src_id" and "tgt_id" exactly match the extracted entity names.'
    input_text = Field(annotation=str required=True json_schema_extra={'desc': 'The text to extract entities and relationships from.', '__dspy_field_type': 'input', 'prefix': 'Input Text:'})
    entity_types = Field(annotation=list[str] 

In [11]:
lm.inspect_history(n=1)




Given a text document that is potentially relevant to this activity and a list of entity types, 
identify all entities of those types from the text and all relationships among the identified entities.

For entities:
    - Each entity name should be an actual atomic word from the input text. Avoid duplicates and generic terms.
    - Make sure descriptions are detailed and comprehensive, including:
        1. The entity's role or significance in the context
        2. Key attributes or characteristics
        3. Relationships to other entities (if applicable)
        4. Historical or cultural relevance (if applicable)
        5. Any notable actions or events associated with the entity
    - All entity types from the text must be included. 
    - IMPORTANT: Only use entity types from the provided 'entity_types' list. Do not introduce new entity types.

For relationships:
    - Make sure relationship descriptions are detailed and comprehensive, including:
        1. The nature of the re

'\n\n\nGiven a text document that is potentially relevant to this activity and a list of entity types, \nidentify all entities of those types from the text and all relationships among the identified entities.\n\nFor entities:\n    - Each entity name should be an actual atomic word from the input text. Avoid duplicates and generic terms.\n    - Make sure descriptions are detailed and comprehensive, including:\n        1. The entity\'s role or significance in the context\n        2. Key attributes or characteristics\n        3. Relationships to other entities (if applicable)\n        4. Historical or cultural relevance (if applicable)\n        5. Any notable actions or events associated with the entity\n    - All entity types from the text must be included. \n    - IMPORTANT: Only use entity types from the provided \'entity_types\' list. Do not introduce new entity types.\n\nFor relationships:\n    - Make sure relationship descriptions are detailed and comprehensive, including:\n        

In [12]:
metrics = [entity_recall_metric, relationship_similarity_metric]
for metric in metrics:
    evaluate = Evaluate(
        devset=devset, 
        metric=metric, 
        num_threads=os.cpu_count(), 
        display_progress=True,
        display_table=5,
    )
    evaluate(miprov2_model)

  0%|          | 0/5 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
Average Metric: 4.06969696969697 / 5  (81.4): 100%|██████████| 5/5 [01:38<00:00, 19.66s/it]  
INFO:dspy.evaluate.evaluate:2024-09-19T07:45:39.675496Z [info     ] Average Metric: 4.06969696969697 / 5 (81.4%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=218


,input_text,example_entities,example_relationships,pred_entities,pred_relationships,entity_recall_metric
0,"Kendrick Lamar’s Pulitzer Prize win is the latest sign of the growing recognition of hip-hop—this time from one of America’s highest-profile cultural institutions. The rapper,...","[{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': ""A renowned rapper who won the Pulitzer Prize for his album 'DAMN.'. He is the first non-classical or jazz...","[{'src_id': 'KENDRICK LAMAR', 'tgt_id': 'PULITZER PRIZE', 'description': ""Kendrick Lamar won the Pulitzer Prize for music for his album 'DAMN.'. This win is significant as it...","[{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': ""A renowned rapper who won the Pulitzer Prize for his album 'DAMN.', marking a significant recognition of hip-hop by...","[{'src_id': 'KENDRICK LAMAR', 'tgt_id': 'PULITZER PRIZE', 'description': ""Kendrick Lamar won the Pulitzer Prize for his album 'DAMN.', marking a significant recognition of hip-hop."", 'weight': 0.9,...",✔️ [1.0]
1,Cramer Remix: The fall of cloud stocks could be a worrisome sign for tech 11 Hours Ago Jim Cramer deciphers the collapse of the “cloud...,"[{'entity_name': 'CRAMER', 'entity_type': 'PERSON', 'description': ""Jim Cramer, a well-known financial commentator and host of 'Mad Money' on CNBC, who is analyzing the fall of cloud...","[{'src_id': 'CRAMER', 'tgt_id': 'CLOUD STOCKS', 'description': 'Jim Cramer is analyzing the fall of cloud stocks, providing insights into what this decline could mean for the...","[{'entity_name': 'JIM CRAMER', 'entity_type': 'PERSON', 'description': 'Jim Cramer is a financial commentator who discusses the fall of cloud stocks and its potential impact on tech...","[{'src_id': 'JIM CRAMER', 'tgt_id': 'CLOUD STOCKS', 'description': 'Jim Cramer discusses the collapse of cloud stocks and its implications.', 'weight': 0.8, 'order': 1}, {'src_id': 'CLOUD STOCKS',...",✔️ [0.6]
2,* Mulberry manufactures 55 pct of leather goods in UK * Import cost of raw materials has gone up after Brexit vote * Brand fully...,"[{'entity_name': 'MULBERRY', 'entity_type': 'ORGANIZATION', 'description': 'A handbag maker committed to manufacturing in Britain, facing higher leather import costs post-Brexit. The company has two manufacturing plants...","[{'src_id': 'MULBERRY', 'tgt_id': 'UK', 'description': 'Mulberry manufactures 55% of its leather goods in the UK and remains committed to the country despite post-Brexit challenges.', 'weight':...","[{'entity_name': 'MULBERRY', 'entity_type': 'ORGANIZATION', 'description': 'A handbag maker committed to manufacturing in Britain, facing higher leather import costs post-Brexit. The company is considering opening another...","[{'src_id': 'MULBERRY', 'tgt_id': 'UK', 'description': 'Mulberry is committed to manufacturing in the UK, producing 55% of its leather goods there.', 'weight': 0.9, 'order': 1}, {'src_id':...",✔️ [0.6363636363636364]
3,"COLUMBIA, Md., April 25, 2018 (GLOBE NEWSWIRE) -- W. R. Grace & Co. (NYSE:GRA) today announced that it has declared a quarterly cash dividend of...","[{'entity_name': 'W. R. GRACE & CO.', 'entity_type': 'ORGANIZATION', 'description': 'A leading global supplier of catalysts and engineered materials, with two industry-leading business segments: Catalysts Technologies...","[{'src_id': 'W. R. GRACE & CO.', 'tgt_id': 'NYSE', 'description': 'W. R. Grace & Co. is listed on the New York Stock Exchange under the symbol...","[{'entity_name': 'W. R. GRACE & CO.', 'entity_type': 'ORGANIZATION', 'description': 'A leading global supplier of catalysts and engineered materials, headquartered in Columbia, Maryland. The company announced...","[{'src_id': 'W. R. GRACE & CO.', 'tgt_id': 'COLUMBIA', 'description': 'W. R. Grace & Co. is headquartered in Columbia, Maryland.', 'weight': 0.4, 'order': 1}, {'src_id': 'W....",✔️ [0.8333333333333334]
4,"Top marketer joins company to lead global marketing and brand strategies FRAMI

  0%|          | 0/5 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/com

,input_text,example_entities,example_relationships,pred_entities,pred_relationships,relationship_similarity_metric
0,"Kendrick Lamar’s Pulitzer Prize win is the latest sign of the growing recognition of hip-hop—this time from one of America’s highest-profile cultural institutions. The rapper,...","[{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': ""A renowned rapper who won the Pulitzer Prize for his album 'DAMN.'. He is the first non-classical or jazz...","[{'src_id': 'KENDRICK LAMAR', 'tgt_id': 'PULITZER PRIZE', 'description': ""Kendrick Lamar won the Pulitzer Prize for music for his album 'DAMN.'. This win is significant as it...","[{'entity_name': 'KENDRICK LAMAR', 'entity_type': 'PERSON', 'description': ""A renowned rapper who won the Pulitzer Prize for his album 'DAMN.', marking a significant recognition of hip-hop by...","[{'src_id': 'KENDRICK LAMAR', 'tgt_id': 'PULITZER PRIZE', 'description': ""Kendrick Lamar won the Pulitzer Prize for his album 'DAMN.', marking a significant recognition of hip-hop."", 'weight': 0.9,...",✔️ [0.75]
1,Cramer Remix: The fall of cloud stocks could be a worrisome sign for tech 11 Hours Ago Jim Cramer deciphers the collapse of the “cloud...,"[{'entity_name': 'CRAMER', 'entity_type': 'PERSON', 'description': ""Jim Cramer, a well-known financial commentator and host of 'Mad Money' on CNBC, who is analyzing the fall of cloud...","[{'src_id': 'CRAMER', 'tgt_id': 'CLOUD STOCKS', 'description': 'Jim Cramer is analyzing the fall of cloud stocks, providing insights into what this decline could mean for the...","[{'entity_name': 'JIM CRAMER', 'entity_type': 'PERSON', 'description': 'Jim Cramer is a financial commentator who discusses the fall of cloud stocks and its potential impact on tech...","[{'src_id': 'JIM CRAMER', 'tgt_id': 'CLOUD STOCKS', 'description': 'Jim Cramer discusses the collapse of cloud stocks and its implications.', 'weight': 0.8, 'order': 1}, {'src_id': 'CLOUD STOCKS',...",✔️ [0.6833333333333332]
2,* Mulberry manufactures 55 pct of leather goods in UK * Import cost of raw materials has gone up after Brexit vote * Brand fully...,"[{'entity_name': 'MULBERRY', 'entity_type': 'ORGANIZATION', 'description': 'A handbag maker committed to manufacturing in Britain, facing higher leather import costs post-Brexit. The company has two manufacturing plants...","[{'src_id': 'MULBERRY', 'tgt_id': 'UK', 'description': 'Mulberry manufactures 55% of its leather goods in the UK and remains committed to the country despite post-Brexit challenges.', 'weight':...","[{'entity_name': 'MULBERRY', 'entity_type': 'ORGANIZATION', 'description': 'A handbag maker committed to manufacturing in Britain, facing higher leather import costs post-Brexit. The company is considering opening another...","[{'src_id': 'MULBERRY', 'tgt_id': 'UK', 'description': 'Mulberry is committed to manufacturing in the UK, producing 55% of its leather goods there.', 'weight': 0.9, 'order': 1}, {'src_id':...",✔️ [0.23333333333333334]
3,"COLUMBIA, Md., April 25, 2018 (GLOBE NEWSWIRE) -- W. R. Grace & Co. (NYSE:GRA) today announced that it has declared a quarterly cash dividend of...","[{'entity_name': 'W. R. GRACE & CO.', 'entity_type': 'ORGANIZATION', 'description': 'A leading global supplier of catalysts and engineered materials, with two industry-leading business segments: Catalysts Technologies...","[{'src_id': 'W. R. GRACE & CO.', 'tgt_id': 'NYSE', 'description': 'W. R. Grace & Co. is listed on the New York Stock Exchange under the symbol...","[{'entity_name': 'W. R. GRACE & CO.', 'entity_type': 'ORGANIZATION', 'description': 'A leading global supplier of catalysts and engineered materials, headquartered in Columbia, Maryland. The company announced...","[{'src_id': 'W. R. GRACE & CO.', 'tgt_id': 'COLUMBIA', 'description': 'W. R. Grace & Co. is headquartered in Columbia, Maryland.', 'weight': 0.4, 'order': 1}, {'src_id': 'W....",✔️ [0.5777777777777778]
4,"Top marketer joins company to lead global marketing

In [13]:
miprov2_model.save(entity_relationship_miprov2_path)

[('extractor.predictor', Predict(StringSignature(input_text, entity_types -> reasoning, entities_relationships
    instructions='Given a text document and a list of entity types, meticulously identify all entities of the specified types from the text. For each entity, provide a comprehensive description that includes its role, key attributes, relationships to other entities, historical or cultural relevance, and notable actions or events. Ensure that only the provided entity types are used and avoid duplicates. For relationships, describe their nature, impact, historical context, evolution, and notable events. Include direct and higher-order relationships up to the third order, ensuring that "src_id" and "tgt_id" exactly match the extracted entity names.'
    input_text = Field(annotation=str required=True json_schema_extra={'desc': 'The text to extract entities and relationships from.', '__dspy_field_type': 'input', 'prefix': 'Input Text:'})
    entity_types = Field(annotation=list[st